In [1]:
!pip install torch
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, BertModel
import torch
from datasets import load_dataset
from tqdm.notebook import tqdm

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
def get_datasets():
  imdb = load_dataset('imdb')
  sst2 = load_dataset('sst2')
  yelp_review_full = load_dataset('yelp_review_full')
  return {'imdb': imdb, 'sst2': sst2, 'yelp': yelp_review_full}, {'imdb': torch.empty((0,768)).to(device),
                                                                  'sst2': torch.empty((0,768)).to(device),
                                                                  'yelp': torch.empty((0,768)).to(device)
                                                                  }

datasets_dict, embedding_dict = get_datasets()

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
model = model.to(device)

In [15]:
for ds_name, dataset in datasets_dict.items():
  print(f'------------ {ds_name} ------------')

  embeddings_tensor = torch.empty((0,768)).to(device)
  outputs = torch.empty((0,768)).to(device)

  torch.save(embeddings_tensor, f'{ds_name}_embeddings.pt')

  for label_sentence in tqdm(dataset['train'], total = len(dataset['train']), leave=False):
    text = label_sentence['text']

    embeddings_tensor = torch.load(f'{ds_name}_embeddings.pt')

    encoded_text = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)

    outputs = model(**encoded_text, output_hidden_states=True).last_hidden_state[:, 0, :]
    #embedding_dict[ds_name] = torch.cat((embedding_dict[ds_name], outputs.last_hidden_state[:, 0, :]))
    embeddings_tensor = torch.cat((embeddings_tensor, outputs), dim=0)

    #print('embedding_dict[ds_name]', embedding_dict[ds_name].element_size() * embedding_dict[ds_name].nelement())

    torch.save(embeddings_tensor, f'{ds_name}_embeddings.pt')

    del encoded_text, outputs
    torch.cuda.empty_cache()


------------ imdb ------------


  0%|          | 0/25000 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [17]:
torch.load('/content/imdb_embeddings.pt').size()

torch.Size([590, 768])